In [1]:
import pandas as pd
import json
import numpy as np
import os
import hashlib
import psycopg
from sqlalchemy import create_engine
from sqlalchemy import URL
from dotenv import load_dotenv

## DB Connection

In [2]:
# Load DB variables
load_dotenv()
DATABASE = os.getenv("PROD_DATABASE")
USER = os.getenv("PROD_USER")
PASSWORD = os.getenv("PROD_PASSWORD")
HOST = os.getenv("PROD_HOST")
PORT = os.getenv("PROD_PORT")

In [3]:
# Connection to DB
url_object = URL.create(
            "postgresql+psycopg",
            username=USER,
            password=PASSWORD,
            host=HOST,
            port=PORT,
            database=DATABASE,
        )

alchemyEngine = create_engine(url_object)

dbConnection = alchemyEngine.connect()

In [4]:
# Reading Tables from DB
df_roll = pd.read_sql('select * from "Rolls"', dbConnection)
df_machine = pd.read_sql('select * from "Machines"', dbConnection)
df_tiles = pd.read_sql('select * from "Tiles"', dbConnection)
df_defects = pd.read_sql('select * from "DefectTiles"', dbConnection)

In [5]:
# Close DB connection
dbConnection.close()

In [6]:
# Modifying defects table
df_defects['ShapePointsX'] = df_defects['ShapePointsX'].replace('{', '[', regex=True).replace('}', ']', regex=True)
df_defects['ShapePointsY'] = df_defects['ShapePointsY'].replace('{', '[', regex=True).replace('}', ']', regex=True)
df_defects = df_defects[df_defects["CreatedBy"] == "operator"]
df_defects.reset_index(drop= True, inplace=True) 

In [7]:
# Save files to local
os.makedirs("../db", exist_ok=True)

path_roll = "../db/rolls.csv"
path_machine = "../db/machines.csv"
path_tiles = "../db/tiles.csv"
path_defects = "../db/defects.csv"

In [8]:
# Save tables as csv
df_roll.to_csv(path_roll)
df_machine.to_csv(path_machine)
df_tiles.to_csv(path_tiles)
df_defects.to_csv(path_defects)